In [1]:
import pandas as pd
import numpy as np

In [2]:
import allel # ref - https://scikit-allel.readthedocs.io/en/stable/io.html#variant-call-format-vcf

In [3]:
vcf_folder = "../dataset/"
vcf_file_freebayes = "syn4-freebayes.vcf.gz"
vcf_file_mutect = "syn4-mutect2.vcf.gz"
vcf_file_vardict = "syn4-vardict.vcf.gz"
vcf_file_varscan = "syn4-varscan.vcf.gz"

In [4]:
df_freebayes = allel.vcf_to_dataframe(vcf_folder + vcf_file_freebayes)

In [5]:
df_freebayes.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10043,.,TA,T,NaN,NaN,57.500000,False
1,1,10045,.,AC,A,NaN,NaN,57.500000,False
2,1,10439,rs112766696,AC,A,NaN,NaN,96.699997,False
3,1,10440,rs112155239,C,A,NaN,NaN,96.699997,False
4,1,14907,rs6682375,A,G,NaN,NaN,589.200012,False


In [6]:
df_mutect = allel.vcf_to_dataframe(vcf_folder + vcf_file_mutect)
df_mutect.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,61499,rs2531301,G,A,NaN,NaN,NaN,False
1,1,61851,rs62637819,T,A,NaN,NaN,NaN,True
2,1,113813,.,A,T,NaN,NaN,NaN,False
3,1,232930,rs146484642,C,G,NaN,NaN,NaN,False
4,1,232955,rs200827137,G,T,NaN,NaN,NaN,False


In [7]:
df_vardict = allel.vcf_to_dataframe(vcf_folder + vcf_file_vardict)
df_vardict.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10340,.,TAACCCTAACCCTACCC,T,NaN,NaN,33.0,False
1,1,10403,.,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC,A,NaN,NaN,61.0,True
2,1,10428,.,CCCTAA,C,NaN,NaN,65.0,True
3,1,10437,.,TA,CT,NaN,NaN,33.0,False
4,1,10439,rs112766696,AC,A,NaN,NaN,29.0,False


In [8]:
df_varscan = allel.vcf_to_dataframe(vcf_folder + vcf_file_varscan)
df_varscan.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10386,.,C,CT,NaN,NaN,0.0,True
1,1,10400,.,C,T,NaN,NaN,0.0,True
2,1,10439,rs112766696,AC,A,NaN,NaN,0.0,False
3,1,10440,rs112155239,C,A,NaN,NaN,0.0,False
4,1,14907,rs6682375,A,G,NaN,NaN,0.0,False


In [9]:
print(f"Real freebayes shape = {df_freebayes.shape}")
print(f"Real Mutect shape = {df_mutect.shape}")
print(f"Real Vardict shape = {df_vardict.shape}")
print(f"Real Varscan shape = {df_varscan.shape}")

Real freebayes shape = (4596475, 9)
Real Mutect shape = (121815, 9)
Real Vardict shape = (4578378, 9)
Real Varscan shape = (4299996, 9)


In [10]:
m1 = pd.merge(df_freebayes, df_mutect, on = ["CHROM", "POS"], how="outer", suffixes = ("_freebayes", "_mutect"))

In [11]:
m1.shape

(4694068, 16)

In [12]:
m2 = pd.merge(m1, df_vardict, on = ["CHROM", "POS"], how="outer")
m2.rename(columns={"ID": "ID_vardict", "REF": "REF_vardict", "ALT_1": "ALT_1_vardict", "ALT_2": "ALT_2_vardict", \
                  "ALT_3": "ALT_3_vardict", "QUAL": "QUAL_vardict", "FILTER_PASS": "FILTER_PASS_vardict"}, inplace=True)

In [13]:
m2.head()

,CHROM,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,FILTER_PASS_freebayes,ID_mutect,...,ALT_3_mutect,QUAL_mutect,FILTER_PASS_mutect,ID_vardict,REF_vardict,ALT_1_vardict,ALT_2_vardict,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict
0,1,10043,.,TA,T,NaN,NaN,57.500000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10045,.,AC,A,NaN,NaN,57.500000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10439,rs112766696,AC,A,NaN,NaN,96.699997,False,NaN,...,NaN,NaN,NaN,rs112766696,AC,A,NaN,NaN,29.0,False
3,1,10440,rs112155239,C,A,NaN,NaN,96.699997,False,NaN,...,NaN,NaN,NaN,rs112155239,C,A,NaN,NaN,33.0,False
4,1,14907,rs6682375,A,G,NaN,NaN,589.200012,False,NaN,...,NaN,NaN,NaN,rs6682375,A,G,NaN,NaN,114.0,False


In [14]:
m2.shape

(4923048, 23)

In [15]:
m3 = pd.merge(m2, df_varscan, on = ["CHROM", "POS"], how="outer")
m3.rename(columns={"ID": "ID_varscan", "REF": "REF_varscan", "ALT_1": "ALT_1_varscan", "ALT_2": "ALT_2_varscan", \
                  "ALT_3": "ALT_3_varscan", "QUAL": "QUAL_varscan", "FILTER_PASS": "FILTER_PASS_varscan"}, inplace=True)
m3.shape

(5045774, 30)

In [16]:
m3.head()

,CHROM,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,FILTER_PASS_freebayes,ID_mutect,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,10043,.,TA,T,NaN,NaN,57.500000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10045,.,AC,A,NaN,NaN,57.500000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10439,rs112766696,AC,A,NaN,NaN,96.699997,False,NaN,...,NaN,29.0,False,rs112766696,AC,A,NaN,NaN,0.0,False
3,1,10440,rs112155239,C,A,NaN,NaN,96.699997,False,NaN,...,NaN,33.0,False,rs112155239,C,A,NaN,NaN,0.0,False
4,1,14907,rs6682375,A,G,NaN,NaN,589.200012,False,NaN,...,NaN,114.0,False,rs6682375,A,G,NaN,NaN,0.0,False


In [17]:
m3.head()[["CHROM", "POS", "FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]]

,CHROM,POS,FILTER_PASS_freebayes,FILTER_PASS_mutect,FILTER_PASS_vardict,FILTER_PASS_varscan
0,1,10043,False,NaN,NaN,NaN
1,1,10045,False,NaN,NaN,NaN
2,1,10439,False,NaN,False,False
3,1,10440,False,NaN,False,False
4,1,14907,False,NaN,False,False


In [18]:
df_merged = m3

In [19]:
# add third category instead of False?
df_merged["FILTER_PASS_freebayes"].fillna(False, inplace=True)
df_merged["FILTER_PASS_vardict"].fillna(False, inplace=True)
df_merged["FILTER_PASS_mutect"].fillna(False, inplace=True)
df_merged["FILTER_PASS_varscan"].fillna(False, inplace=True)
df_merged.head()[["CHROM", "POS", "FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]]

,CHROM,POS,FILTER_PASS_freebayes,FILTER_PASS_mutect,FILTER_PASS_vardict,FILTER_PASS_varscan
0,1,10043,False,False,False,False
1,1,10045,False,False,False,False
2,1,10439,False,False,False,False
3,1,10440,False,False,False,False
4,1,14907,False,False,False,False


In [20]:
truth = pd.read_csv(vcf_folder + "syn4_truth.bed", sep = "\t", header = None)
truth.columns = ["CHROM", "POS_START", "POS_END"]
truth.head()

,CHROM,POS_START,POS_END
0,1,768126,768126
1,1,1027828,1027828
2,1,1202003,1202003
3,1,1212160,1212160
4,1,1354499,1354499


In [21]:
(truth.POS_START == truth.POS_END).sum()

16315

In [22]:
truth.shape

(16315, 3)

In [23]:
truth.merge(df_merged, left_on = ["CHROM", "POS_START"], right_on = ["CHROM", "POS"])
# this is smaller than the actual truth file indicating the all 4 variant callers missed to classify some true variants

,CHROM,POS_START,POS_END,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,768126,768126,768126,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,.,T,G,NaN,NaN,0.0,True
1,1,1027828,1027828,1027828,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30.0,False,.,T,G,NaN,NaN,0.0,True
2,1,1202003,1202003,1202003,.,C,G,NaN,NaN,6.0,...,NaN,53.0,True,NaN,NaN,NaN,NaN,NaN,NaN,False
3,1,1212160,1212160,1212160,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,70.0,True,.,T,G,NaN,NaN,0.0,True
4,1,1354499,1354499,1354499,.,C,T,NaN,NaN,34.0,...,NaN,62.0,True,.,C,T,NaN,NaN,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15387,Y,23487532,23487532,23487532,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,87.0,True,.,A,C,NaN,NaN,0.0,False
15388,Y,23501006,23501006,23501006,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,52.0,True,NaN,NaN,NaN,NaN,NaN,NaN,False
15389,Y,23826515,23826515,23826515,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,99.0,True,.,C,T,NaN,NaN,0.0,False
15390,Y,24396542,24396542,24396542,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,66.0,True,.,A,T,NaN,NaN,0.0,True


In [24]:
df_merged_with_truth = pd.merge(truth, df_merged, left_on=["CHROM", "POS_START"], right_on = ["CHROM", "POS"], how="outer")
df_merged_with_truth.head()

,CHROM,POS_START,POS_END,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,768126.0,768126.0,768126.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,.,T,G,NaN,NaN,0.0,True
1,1,1027828.0,1027828.0,1027828.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30.0,False,.,T,G,NaN,NaN,0.0,True
2,1,1202003.0,1202003.0,1202003.0,.,C,G,NaN,NaN,6.0,...,NaN,53.0,True,NaN,NaN,NaN,NaN,NaN,NaN,False
3,1,1212160.0,1212160.0,1212160.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,70.0,True,.,T,G,NaN,NaN,0.0,True
4,1,1354499.0,1354499.0,1354499.0,.,C,T,NaN,NaN,34.0,...,NaN,62.0,True,.,C,T,NaN,NaN,0.0,True


In [25]:
df_merged_with_truth.shape

(5046697, 32)

In [26]:
def set_true_label(row):
    if str(row) == "nan":
        return False
    else:
        return True

In [27]:
df_merged_with_truth["truth"] = df_merged_with_truth["POS_START"].apply(lambda row:set_true_label(row))

In [28]:
extras_in_truth = df_merged_with_truth[df_merged_with_truth.POS.isna()].index
extras_in_truth

Int64Index([    6,     9,    16,    18,    21,    27,    30,    31,    53,
               65,
            ...
            15720, 15773, 15820, 15851, 15908, 16109, 16195, 16230, 16250,
            16302],
           dtype='int64', length=923)

In [29]:
df_merged_with_truth.drop(extras_in_truth, inplace=True)

In [30]:
df_merged_with_truth.isna().sum()

CHROM                          0
POS_START                5030382
POS_END                  5030382
POS                            0
ID_freebayes              449299
REF_freebayes             449299
ALT_1_freebayes           449299
ALT_2_freebayes          4983672
ALT_3_freebayes          5043168
QUAL_freebayes            449300
FILTER_PASS_freebayes          0
ID_mutect                4923908
REF_mutect               4923908
ALT_1_mutect             4923908
ALT_2_mutect             5045774
ALT_3_mutect             5045774
QUAL_mutect              5045774
FILTER_PASS_mutect             0
ID_vardict                450229
REF_vardict               450229
ALT_1_vardict             450229
ALT_2_vardict            5045774
ALT_3_vardict            5045774
QUAL_vardict              450250
FILTER_PASS_vardict            0
ID_varscan                733186
REF_varscan               733186
ALT_1_varscan             733186
ALT_2_varscan            5045774
ALT_3_varscan            5045774
QUAL_varsc

In [31]:
df_merged_with_truth["QUAL_freebayes"].fillna(df_merged_with_truth["QUAL_freebayes"].mean(), inplace=True)
df_merged_with_truth["QUAL_vardict"].fillna(df_merged_with_truth["QUAL_vardict"].mean(), inplace=True)

In [32]:
df_merged_with_truth["QUAL_mutect"].isna().sum()

5045774

In [33]:
df_merged_with_truth[["QUAL_freebayes", "QUAL_vardict"]].corr()

,QUAL_freebayes,QUAL_vardict
QUAL_freebayes,1.000000,0.479862
QUAL_vardict,0.479862,1.000000


#### Confusion matrix comparing against truth

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_freebayes)).ravel()
tn, fp, fn, tp

(5011124, 19258, 6313, 9079)

In [36]:
# least false positives
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_mutect)).ravel()
tn, fp, fn, tp

(5013027, 17355, 2787, 12605)

In [37]:
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_vardict)).ravel()
tn, fp, fn, tp

(4999962, 30420, 2587, 12805)

In [38]:
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_varscan)).ravel()
tn, fp, fn, tp

(4971699, 58683, 3460, 11932)

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_merged_with_truth[["FILTER_PASS_mutect", "FILTER_PASS_freebayes", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]], df_merged_with_truth["truth"], test_size = 0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4036619, 4)
(1009155, 4)
(4036619,)
(1009155,)


In [40]:
from pgmpy.models import BayesianNetwork

In [41]:
bn = BayesianNetwork(
[
#     ("CHROM", "truth"),
#     ("POS", "truth"),
#     ("QUAL_freebayes", "truth"),
#     ("QUAL_vardict", "truth"),
    ("FILTER_PASS_mutect", "truth"),
    ("FILTER_PASS_freebayes", "truth"),
    ("FILTER_PASS_vardict", "truth"),
    ("FILTER_PASS_varscan", "truth"),
])

In [42]:
from pgmpy.estimators import MaximumLikelihoodEstimator

In [43]:
train_df = pd.DataFrame(X_train, columns = ["FILTER_PASS_mutect","FILTER_PASS_freebayes", "FILTER_PASS_vardict", "FILTER_PASS_varscan"])
train_df["truth"] = y_train
print(train_df.shape)
train_df.head()

(4036619, 5)


,FILTER_PASS_mutect,FILTER_PASS_freebayes,FILTER_PASS_vardict,FILTER_PASS_varscan,truth
543750,False,False,False,False,False
4042760,False,False,False,False,False
215429,False,False,False,False,False
80497,False,False,False,False,False
334109,False,False,False,False,False


In [44]:
bn.fit(
    data=train_df,
    estimator=MaximumLikelihoodEstimator
)

In [45]:
print(bn.get_cpds("truth"))

+-----------------------+-----+-----------------------------+
| FILTER_PASS_freebayes | ... | FILTER_PASS_freebayes(True) |
+-----------------------+-----+-----------------------------+
| FILTER_PASS_mutect    | ... | FILTER_PASS_mutect(True)    |
+-----------------------+-----+-----------------------------+
| FILTER_PASS_vardict   | ... | FILTER_PASS_vardict(True)   |
+-----------------------+-----+-----------------------------+
| FILTER_PASS_varscan   | ... | FILTER_PASS_varscan(True)   |
+-----------------------+-----+-----------------------------+
| truth(False)          | ... | 0.32415208468351697         |
+-----------------------+-----+-----------------------------+
| truth(True)           | ... | 0.675847915316483           |
+-----------------------+-----+-----------------------------+


In [46]:
test_df = pd.DataFrame(X_test, columns = ["FILTER_PASS_mutect","FILTER_PASS_freebayes", "FILTER_PASS_vardict", "FILTER_PASS_varscan"])
test_df["truth"] = y_test
print(test_df.shape)
test_df.head()

(1009155, 5)


,FILTER_PASS_mutect,FILTER_PASS_freebayes,FILTER_PASS_vardict,FILTER_PASS_varscan,truth
3100631,False,False,False,False,False
725263,False,False,False,False,False
363377,False,False,False,False,False
2015143,False,False,False,False,False
1519730,False,False,False,False,False


In [47]:
from pgmpy.inference import VariableElimination
from ipywidgets import FloatProgress

In [48]:
bn_infer = VariableElimination(bn)

In [49]:
from IPython.display import clear_output

In [50]:
def get_preds(row):
    prob = bn_infer.query(variables=["truth"], evidence={"FILTER_PASS_freebayes": row["FILTER_PASS_freebayes"], \
                                             "FILTER_PASS_mutect": row["FILTER_PASS_mutect"], \
                                             "FILTER_PASS_vardict": row["FILTER_PASS_vardict"], \
                                             "FILTER_PASS_varscan": row["FILTER_PASS_varscan"]}, show_progress=False)
#     clear_output(wait=True)
    if (prob.values[1]) > 0.5:
        return True
    else:
        return False

In [51]:
test_df["preds"] = test_df.apply(get_preds, axis = 1)

#### Metric evaluation

In [52]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [53]:
from sklearn.metrics import classification_report

In [54]:
print(classification_report(test_df["truth"], test_df["preds"]))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00   1006157
        True       0.63      0.67      0.65      2998

    accuracy                           1.00   1009155
   macro avg       0.82      0.83      0.82   1009155
weighted avg       1.00      1.00      1.00   1009155

